In [ ]:
from PIL import Image

import pytesseract

In [10]:
import pandas as pd

df = pd.DataFrame(columns=["SMS"])

In [11]:
df

,SMS


In [ ]:
import os

files = os.listdir("./spam/new")

for file in files:
    print(file)
    path = "./spam/new/" + file
    if os.path.isfile(path):
        df.loc[len(df)] = [pytesseract.image_to_string(Image.open(path))]

aides_villes.webp
ameli.png
ameli2.jpg
ameli3.jpg
amende.png
amende2.png
amende3.png
banque.jpg
banque2.png
banque3.jpg
banque4.jpg
colis.jpg
colis.png
colis10.jpg
colis11.png
colis12.png
colis13.jpg
colis14.jpg
colis1482.jpg
colis15.png
colis150.jpg
colis2.jpg
colis3.jpg
colis4.jpg
colis5.jpg
colis6.jpg
colis7.jpg
colis8.jpg
colis9.jpg
cpf.png
crit_air.png
formation.jpg
google.jpg
lien-sms-fraude.jpg
netflix.png
netflx2.png
nickel.jpg
phishing-colis.jpg
prime_carburant.jpg
rdv_medical.png
sentiment.jpg
sfr2.png
ubereats.png
ulys.jpg
ulys2.jpg
ulys3.jpg
ulys4.jpg
ulys5.jpg
ulys6.jpg
ulys7.jpg


# Règles

## Amende
Le gouvernement n'envoie pas d'amendes par SMS.
Le site officiel pour régler ses contraventions : www.amendes.gouv.fr

## Colis
Les URL officielles:
- **Chronopost** : www.chronopost.fr
- **Colissimo** : www.laposte.fr
- **DHL** : www.dhl.com

## Ameli
www.ameli.fr

L’Assurance Maladie ne vous demandera jamais :
- vos identifiants de connexion au compte ameli, notamment votre mot de passe ;
- des informations médicales ;
- vos coordonnées bancaires complètes ou votre numéro de carte bancaire.

## Regex

In [1]:
import pandas as pd

df = pd.read_csv("./data/test_ocr.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   SMS     114 non-null    object
dtypes: object(1)
memory usage: 1.0+ KB


In [75]:
import re

patterns_ortho = [
    re.compile(r'[a-z][!?:;]'),
    # re.compile(r'^[a-z]'),
    re.compile(r'\w[!?:;]{2,}'),
    re.compile(r'\b[A-Z][a-z]+[A-Z]\w+\b'),
    re.compile(r'\bvou\b(?!s)'), 
    re.compile(r'\bvotre\b.*\beste\b'), 
    re.compile(r'\bà\b.*\bétait\b')
]

patterns_gift = [
    re.compile(r'\bgagn[éeè]\b'),
    re.compile(r'\bcadeau\b'),
    re.compile(r'\bprix\b'),
    re.compile(r'\bloterie\b'),
    re.compile(r'\bfelicitation\b'),
    re.compile(r'\brécompense\b'),
    re.compile(r'\bgratuit\b'),
    re.compile(r'\bcliqu(ez|er)\b.*\blien\b'),
    re.compile(r'\bréclam(ez|er)\b'),
    re.compile(r'\bobten(ez|er)\b'),
    re.compile(r'\btirage au sort\b'),
    re.compile(r'\bheureux gagnant\b'),
]

patterns_paiement = [
    re.compile(r'\bpa(y|i)(ez|er|ement)\b'),
    re.compile(r'\bfrais\b'),
    re.compile(r'\bfacture\b'),
    re.compile(r'\brégl(ez|er)\b'),
    re.compile(r'\bdébloqu(ez|er)\b'),
    re.compile(r'\bsuspend(u|re)\b.*\bcompte\b'),
    re.compile(r'\bmaintenir.*\bcompte\b'),
    re.compile(r'\bactiv(ez|er).*\bcompte\b'),
    re.compile(r'\b\d+[€$£]\b.*\b(urgent|immédiat|maintenant)\b'),  
    re.compile(r'\bamende\b'),
    re.compile(r'\bpr(eéè)l(éeè)vement\b'),
    re.compile(r'contravention')
]

patterns_colis = [
    re.compile(r'\bcolis\b'),
    re.compile(r'\blivraison\b.*\bpay'),
]

patterns_info = [
    re.compile(r'\bRIB\b'),
    re.compile(r'\bIBAN\b'),
    re.compile(r'\bcarte bancaire\b'),
    re.compile(r'\bcode\b.*\bcarte\b'),
    re.compile(r'\bcode secret\b'),
    re.compile(r'\bPIN\b'),
    re.compile(r'\bCVV\b'),
    re.compile(r'\bcryptogramme\b'),
    re.compile(r'\bidentifiant\b'),
    re.compile(r'\bmot de passe\b'),
    re.compile(r'\b2FA\b'),
    re.compile(r'\bOTP\b'),
    re.compile(r'\bcarte d.identit[ée]\b'),
    re.compile(r'\bpasseport\b'),
    re.compile(r'\bpièce d.identité\b'),
    re.compile(r'\bconfirm(ez|er).*\b(identité|compte|informations)\b'),
    re.compile(r'\bverifi(ez|er).*\b(identité|compte)\b'),
    re.compile(r'\bmettre à jour.*\b(coordonnées|informations)\b'),
]


In [83]:
def check_pattern(pattern, text):
    for p in pattern:
        if p.search(text, re.IGNORECASE):  
            return True
    return False

def check_all_patterns(text):
    if check_pattern(patterns_paiement, text):
        return "Demande de paiement suspect"
    if check_pattern(patterns_colis, text):
        return "Colis"
    # elif check_pattern(patterns_ortho, text):
    #     return "Fautes ou formulation incohérente"
    elif check_pattern(patterns_info, text):
        return "Demande d'informations sensibles"
    elif check_pattern(patterns_gift, text):
        return "Cadeau piégé"


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   SMS     114 non-null    object
dtypes: object(1)
memory usage: 1.0+ KB


In [84]:
df['SMS'] = df['SMS'].fillna("")
df['Spam'] = df['SMS'].apply(check_all_patterns)

In [85]:
df.to_csv("./data/with_spam.csv")